# Code de régression de Keras dans le cadre du projet 

Appliquons maintenant nos connaissances à un ensemble de données plus réalistes. Nous allons également nous concentrer sur l'ingénierie des caractéristiques (feature engineering) et le nettoyage de nos données (data cleaning) !

## Les données

Nous utiliserons des données provenant d'un ensemble de données Kaggle :

https://www.kaggle.com/harlfoxem/housesalesprediction

#### Colonnes Features (caractéristiques)
    
* id - ID unique pour chaque maison vendue
* date - Date de la vente de la maison
* price - Prix de chaque maison vendue
* bedrooms - Nombre de chambres
* bathrooms - Nombre de salles de bains, où 0,5 correspond à une chambre avec toilettes mais sans douche
* sqft_living - Superficie habitable
* sqft_lot - Superficie du terrain
* floors - Nombre d'étages
* waterfront - Une variable dummy (fictive) indiquant si l'appartement donnait sur le front de mer ou non
* view - Un indice de 0 à 4 de la qualité de la vue de la propriété
* condition - Un indice de 1 à 5 sur l'état de l'appartement
* grade - Un indice de 1 à 13, où 1-3 a un niveau de qualité faible de construction et de conception des bâtiments, 7 a un niveau moyen de construction et de conception, et 11-13 a un niveau de qualité élevé de construction et de conception
* sqft_above - La superficie de l'espace intérieur du logement qui est au-dessus du niveau du sol
* sqft_basement - La superficie de l'espace intérieur du logement qui se trouve sous le niveau du sol
* yr_built - L'année où la maison a été initialement construite
* yr_renovated - L'année de la dernière rénovation de la maison
* zipcode - Dans quelle zone de code postal se trouve la maison
* lat - Latitude
* long - Longitude
* sqft_living15 - La superficie de l'espace intérieur de l'habitation pour les 15 voisins les plus proches
* sqft_lot15 - La superficie du terrains des 15 voisins les plus proches

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('kc_house_data.csv')



## Analyse Exploratoire des Données
### (Exploratory Data Analysis - EDA)

In [ ]:
df.isnull().sum()

In [ ]:
df.describe().transpose()

In [ ]:
plt.figure(figsize=(12,6))
sns.histplot(df['price']);

In [ ]:
sns.countplot(x=df['bedrooms']);

In [ ]:
df.corr()

In [ ]:
df.corr()['price'].sort_values()

In [ ]:
plt.figure(figsize=(12,8))
sns.scatterplot(x='price',y='sqft_living',data=df);

In [ ]:
plt.figure(figsize=(12,8))
sns.boxplot(x='bedrooms',y='price',data=df);

### Propriétés géographiques

In [ ]:
plt.figure(figsize=(12,8))
sns.scatterplot(x='price',y='long',data=df)

In [ ]:
plt.figure(figsize=(12,8))
sns.scatterplot(x='price',y='lat',data=df);

In [ ]:
plt.figure(figsize=(12,8))
sns.scatterplot(x='long',y='lat',data=df,hue='price');

In [ ]:
df.sort_values('price',ascending=False).head(20)

In [ ]:
len(df)*(0.01)

In [ ]:
non_top_1_perc = df.sort_values('price',ascending=False).iloc[216:]

In [ ]:
plt.figure(figsize=(12,8))
sns.scatterplot(x='long',y='lat',
                data=non_top_1_perc,hue='price',
                palette='RdYlGn',edgecolor=None,alpha=0.2);

### Autres Features

In [ ]:
plt.figure(figsize=(12,8))
sns.boxplot(x='waterfront',y='price',data=df);

### Travailler avec les données de Feature

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df = df.drop('id',axis=1)

In [ ]:
df.head()

### Feature Engineering à partir de la date

In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
df['month'] = df['date'].apply(lambda date:date.month)

In [ ]:
df['year'] = df['date'].apply(lambda date:date.year)

In [ ]:
sns.boxplot(x='year',y='price',data=df);

In [ ]:
plt.figure(figsize=(12,8))
sns.boxplot(x='month',y='price',data=df);

In [ ]:
df.groupby('month').mean()['price'].plot();

In [ ]:
df.groupby('year').mean()['price'].plot();

In [ ]:
df = df.drop('date',axis=1)

In [ ]:
df.columns

In [ ]:

# Il pourrait être utile d'envisager de la supprimer cette catégorie 
# ou de faire du feature engineering de catégories
df['zipcode'].value_counts()

In [ ]:
df = df.drop('zipcode',axis=1)

In [ ]:
df.head()

In [ ]:
# cela pourrait avoir du sens en raison de l'échelle,
# une valeur plus élevée devrait être corrélée à une valeur plus élevée
df['yr_renovated'].value_counts()

In [ ]:
df['sqft_basement'].value_counts()

## Mise à l'échelle et Répartition Entraînement / Test

In [ ]:
X = df.drop('price',axis=1).values
y = df['price'].values

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=101)

### Mise à l'échelle (Scaling)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()

In [ ]:
X_train= scaler.fit_transform(X_train)

In [ ]:
X_test = scaler.transform(X_test)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

## Création du modèle

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam

In [ ]:
model = Sequential()

model.add(Dense(19,activation='relu'))
model.add(Dense(19,activation='relu'))
model.add(Dense(19,activation='relu'))
model.add(Dense(19,activation='relu'))
model.add(Dense(1))

model.compile(optimizer='adam',loss='mse')

## Entraînement du modèle

In [ ]:
model.fit(x=X_train,y=y_train,
          validation_data=(X_test,y_test),
          batch_size=128,epochs=400)

In [ ]:
losses = pd.DataFrame(model.history.history)
losses

In [ ]:
losses.plot();

## Évaluation sur les données Test

https://scikit-learn.org/stable/modules/model_evaluation.html#regression-metrics

In [ ]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,explained_variance_score

### Prédiction sur de toutes nouvelles données

In [ ]:
X_test

In [ ]:
predictions = model.predict(X_test)

In [ ]:
mean_absolute_error(y_test,predictions)

In [ ]:
np.sqrt(mean_squared_error(y_test,predictions))

In [ ]:
explained_variance_score(y_test,predictions)

In [ ]:
df['price'].mean()

In [ ]:
df['price'].median()

In [ ]:
# Nos prédictions
plt.scatter(y_test,predictions)

# Prédictions parfaites
plt.plot(y_test,y_test,'r');

In [ ]:
errors = y_test.reshape(6480, 1) - predictions

In [ ]:
sns.displot(errors);

-------------
### Prédiction sur une maison toute neuve

In [ ]:
single_house = df.drop('price',axis=1).iloc[0]

In [ ]:
single_house = scaler.transform(single_house.values.reshape(-1, 19))

In [ ]:
single_house

In [ ]:
model.predict(single_house)

In [ ]:
df.iloc[0]